# Analyse Quantitative d'Images : Extraction de Mesures

L'analyse d'images est le processus d'extraction d'informations quantitatives et significatives à partir de données visuelles. Au-delà de la simple manipulation, l'objectif est de mesurer des caractéristiques spécifiques des objets présents dans l'image. Par exemple, en biologie, cela peut inclure le comptage de cellules, la mesure de leur taille ou la détermination de leur position.

Ce chapitre présente un pipeline d'analyse d'images typique, de la segmentation à l'extraction de propriétés.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from scipy import ndimage


---

## Image d'exemple : cellules

Nous utiliserons une image de cellules fournie par la bibliothèque `scikit-image` pour nos démonstrations.

In [ ]:
image_originale = data.cells3d()[:, 1, :, :] # Extraction d'une tranche 2D d'une image 3D
image_originale = image_originale[0] # Sélection du premier canal (si applicable)

plt.figure(figsize=(6, 6))
plt.imshow(image_originale, cmap='gray')
plt.title("Image Originale de Cellules")
plt.axis('off')
plt.show()

---

## 1. Segmentation par seuillage

La **segmentation** est la première étape cruciale de l'analyse d'images, consistant à isoler les objets d'intérêt de l'arrière-plan. Pour les images présentant un contraste suffisant, un simple **seuillage** est une méthode efficace. Cette technique crée une image binaire (un "masque") où les pixels dont l'intensité dépasse un seuil sont marqués comme actifs (par exemple, blanc ou `True`), et les autres comme inactifs (noir ou `False`).

In [ ]:
# Définition d'un seuil (peut être ajusté expérimentalement)
seuil = 70
masque_binaire = image_originale > seuil

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(image_originale, cmap='gray')
axes[0].set_title("Originale")
axes[0].axis('off')

axes[1].imshow(masque_binaire, cmap='gray')
axes[1].set_title(f"Masque Binaire (Seuil > {seuil})")
axes[1].axis('off')
plt.show()

---

## 2. Nettoyage morphologique

Les masques binaires obtenus par seuillage peuvent contenir du bruit (petits objets indésirables, trous). Les **opérations morphologiques** sont utilisées pour affiner ces masques. L'**ouverture binaire** (`ndimage.binary_opening`) est une opération qui supprime les petits objets et les connexions fines, contribuant ainsi à éliminer le bruit et à séparer les objets adjacents.

In [ ]:
# Application de l'ouverture binaire pour nettoyer le masque
masque_nettoye = ndimage.binary_opening(masque_binaire, iterations=2)

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(masque_binaire, cmap='gray')
axes[0].set_title("Masque Binaire Brut")
axes[0].axis('off')

axes[1].imshow(masque_nettoye, cmap='gray')
axes[1].set_title("Masque Nettoyé (Ouverture)")
axes[1].axis('off')
plt.show()

---

## 3. Étiquetage des Objets

Une fois le masque nettoyé, l'étape suivante consiste à identifier chaque objet distinct. La fonction `ndimage.label()` attribue un numéro (une "étiquette") unique à chaque composante connectée de pixels actifs dans l'image binaire, permettant ainsi de les traiter individuellement.

In [ ]:
# Étiquetage des objets connectés
objets_etiquetes, nb_objets = ndimage.label(masque_nettoye)

print(f"Nombre d'objets (cellules) détectés : {nb_objets}")

plt.figure(figsize=(6, 6))
# Utilisation d'une colormap pour distinguer visuellement chaque objet étiqueté
plt.imshow(objets_etiquetes, cmap='nipy_spectral')
plt.title("Objets Étiquetés")
plt.axis('off')
plt.show()

---

## 4. Extraction de Propriétés : Centre de Masse

Après l'étiquetage, diverses propriétés peuvent être extraites pour chaque objet. Le **centre de masse** (ou centroïde) est une propriété courante qui indique la position moyenne des pixels d'un objet. La fonction `ndimage.center_of_mass()` calcule cette propriété pour tous les objets étiquetés.

In [ ]:
# Génération des indices des objets pour la fonction center_of_mass
indices_objets = np.arange(1, nb_objets + 1)
centres_de_masse = ndimage.center_of_mass(image_originale, objets_etiquetes, indices_objets)

print(f"Coordonnées des centres de masse des {len(centres_de_masse)} premiers objets :")
for i, centre in enumerate(centres_de_masse[:5]):
    print(f"  Objet {i+1}: (ligne={centre[0]:.1f}, colonne={centre[1]:.1f})")

---

## 5. Visualisation des Résultats

La visualisation des résultats est une étape essentielle pour valider l'analyse. Nous allons superposer les centres de masse détectés sur l'image originale pour confirmer leur positionnement.

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(image_originale, cmap='gray')

# Extraction des coordonnées X (colonnes) et Y (lignes) des centres
y_coords = [c[0] for c in centres_de_masse]
x_coords = [c[1] for c in centres_de_masse]

# Superposition des centres de masse sous forme de points rouges
plt.scatter(x_coords, y_coords, c='red', s=10) # s définit la taille des marqueurs

plt.title(f"Analyse d'Image : {nb_objets} Objets Détectés")
plt.axis('off')
plt.show()